<a href="https://colab.research.google.com/github/aj4abinjacob/Google-Maps-Review-Scraper/blob/main/Google_maps_review_scrapper_Abin_Jacob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing libraries and packages in ubuntu
!sudo apt update
!sudo apt-get upgrade
!apt install chromium-chromedriver
!pip install selenium
!pip install selenium-wire
!sudo apt install openssl
!pip install --upgrade requests

In [2]:
import pandas as pd
import requests,json,time,random
from selenium.webdriver.common.by import By
import sys
from seleniumwire.utils import decode
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from seleniumwire import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
timeout = 5

In [3]:
def round( n ):
    a = (n // 10) * 10
    b = a + 10
    return (a if a == n else b)


#Get the google maps api url
def getGoogleMapsApiURL(url):
  driver = webdriver.Chrome('chromedriver',options=options)
  site = driver.get(url)
  time.sleep(4)
  
 
  # Get total number of reviews

  number_of_reviews = driver.find_element(by=By.CSS_SELECTOR,value='div[jsan="7.fontBodySmall"]').text

  number_of_reviews = int(''.join(filter(str.isdigit, number_of_reviews)))
  number_of_calls = round(number_of_reviews)
  print(f"Getting Url...")
  # driver.save_screenshot("image.png")
  # Sort by recent
  sort_btn = driver.find_element(by=By.CSS_SELECTOR,value='button[aria-label="Sort reviews"]')
  sort_btn.click()
  time.sleep(random.uniform(1, 2))
  newest = driver.find_element(by=By.CSS_SELECTOR,value='ul[role = "menu"] li:nth-child(2)')
  newest.click()
  time.sleep(random.uniform(1, 3))

  #Find scroll layout
  place_rating = driver.find_element(by=By.CLASS_NAME,value="fontDisplayLarge").text
  try:
    scrollable_div = driver.find_element(by=By.CSS_SELECTOR, value = 'div[data-review-id] div[data-review-id]')
  except:
    scrollable_div = driver.find_element(by=By.CSS_SELECTOR, value = 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')

  

  #Scroll Once
  last_element = ""
  for i in range(0,2):
          driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
                  scrollable_div)

  #Get the api url from driver logs 
  api_url = "not found"
  headers = "not found"
  for item in driver.requests:
       if "listentitiesreviews" in str(item):
        api_url = str(item)
        headers = dict(item.headers)  
        headers = (headers)  
        break
  driver.find_element(by=By.CSS_SELECTOR,value='button[aria-label="Back"]').click()
  time.sleep(random.uniform(1, 2))
  place_info = {}
  place_name = driver.find_element(by=By.TAG_NAME,value = "h1").text
  place_info["Place Name"] = place_name
  print(f"Strating crawling for {place_name}")
  print(f"{number_of_reviews} reviews has been found.")
  place_info["Place Rating"] = place_rating
  for el in ["Copy address","Open website","Copy phone number","Copy plus code",'button[jsaction="pane.rating.category"]','div[role="region"][aria-label^="About"]']:
    try:
      if "Copy" in el:
        place_info[el.split("Copy")[1].title()] = driver.find_element(by=By.CSS_SELECTOR,value=f'button[data-tooltip="{el}"]').text
      elif "category" in el:
        place_info['Category'] = driver.find_element(by=By.CSS_SELECTOR,value=el).text
      elif "About" in el:
        place_info["About"] = driver.find_element(by=By.CSS_SELECTOR,value=el).text
      else:
        web_info = driver.find_elements(by=By.CSS_SELECTOR,value=f'button[data-tooltip="{el}"]')[0].get_attribute("aria-label")
        place_info[el.split("Open")[1].title()] = web_info.split("Website: ")[1]
    except:
      pass
  return api_url,number_of_calls,headers,place_info



#Get Review df
def getGoogleMapsReviews(url,date_limit="Unlimited"):
  api_url,number_of_calls,HEADERS,place_info = getGoogleMapsApiURL(url)
  print(place_info)
  if api_url == "not found":
    print("Could not fetch api url :( , try after sometime")
  else:
    df_data_list = []
    last_item = []
    j = 0
    for call in range(0,number_of_calls,10):
      response_data = requests.get(api_url.replace("!1i0!",f"!1i{call}!"),headers=HEADERS)
      # print(response_data.text)
      try:
        response_json = response_data.text.split(")]}'")[1]
      except:
        j = 1
      if j == 1:
        break
      print(f"\rCrawling {int(len(df_data_list)/10)+1} page{'.'*random.randint(2,4)}",end="")
      for data in json.loads(response_json)[2]:
        # print(data)
        try:
          author = data[0][1]
          date = data[27]
          date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(date/1000.0))
          if date_limit != "Unlimited" and date_limit > date:
            print(f"\nCurrent review date : {date} ***** User set date limit : {date_limit}")
            j = 1
            print("Date set has been reached")
            break

          review = data[3]
          rating = data[4]
          current_item = [author,date,review,rating]
          df_data_list.append(current_item)
        except Exception as e:
          print(e)
          print(f"Loaded {len(df_data_list)} reviews but got captcha issues")
          j = 1
      time.sleep(random.uniform(1,3))
    df = pd.DataFrame(df_data_list,columns=["Author","date","Review","Review Rating"])
    if df.shape[0] > 0:
      df['Url'] = url
      for key in place_info:
        df[key] = place_info[key]
      print(f"{df.shape[0]} reviews crawled")
      print(f"Crawl Completed for {place_info['Place Name']}\n")

      return df 
    else:
      print("No reviews crawled for",url)

  

In [6]:
df = getGoogleMapsReviews(url = "https://www.google.com/maps/place/Hoover+Dam/@36.0160655,-114.7399212,17z/data=!4m8!3m7!1s0x80c92b497f82a14b:0x89d59d0bd29de37!8m2!3d36.0160655!4d-114.7377325!9m1!1b1!16zL20vMDNxNGs?authuser=0&hl=en",date_limit = "2022-01-01")
df

Getting Url...
Strating crawling for Hoover Dam
46702 reviews has been found.
{'Place Name': 'Hoover Dam', 'Place Rating': '4.7', ' Address': 'Nevada 89005', ' Website': 'usbr.gov ', ' Phone Number': '(702) 494-2517', ' Plus Code': '2786+CW Boulder City, Nevada', 'Category': 'Hydroelectric power plant', 'About': 'Depression-era engineering marvel offering tours & scenic views of the Colorado River & Lake Mead.'}
Crawling 171 page...
Current review date : 2021-12-31 23:54:44 ***** User set date limit : 2022-01-01
Date set has been reached
1702 reviews crawled
Crawl Completed for Hoover Dam



,Author,date,Review,Review Rating,Url,Place Name,Place Rating,Address,Website,Phone Number,Plus Code,Category,About
0,Jorge González Magallón,2022-05-11 04:58:47,"Impresionante obra de ingeniería, requería sis...",5,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
1,Berlino,2022-05-11 04:19:44,None,5,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
2,Raja N,2022-05-11 02:49:44,What an amazing place. You see such huge dam i...,5,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
3,Nelson Perryman,2022-05-10 23:53:09,Hoover Dam was a wonderful place to visit. T...,5,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
4,Simon Röwer,2022-05-10 21:57:19,None,5,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,Gokulbabu K.G,2022-01-01 02:46:05,None,5,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
1698,J R,2022-01-01 01:19:27,"extremely trafficked, tiny roads, but still wo...",4,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
1699,Mikey Recio,2022-01-01 00:26:42,None,3,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
1700,Mohammed Alqahtani,2022-01-01 00:19:55,Amazing,5,https://www.google.com/maps/place/Hoover+Dam/@...,Hoover Dam,4.7,Nevada 89005,usbr.gov,(702) 494-2517,"2786+CW Boulder City, Nevada",Hydroelectric power plant,Depression-era engineering marvel offering tou...
